In [ ]:
!pip install ultralytics supervision opencv-python deep_sort_realtime

In [2]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import supervision as sv
import numpy as np


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
# Load the YOLOv8 model (IR-trained)
model = YOLO('/content/drive/MyDrive/VIP CUP/drone_tracking_model_new_1.pt')


In [5]:
tracker = DeepSort(max_age=30, n_init=3, nms_max_overlap=1.0, embedder="mobilenet", half=True)


In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import time
import numpy as np

def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea + 1e-6)
    return iou

track_last_seen = dict()
track_missed = dict()
track_prev_size = dict()
track_direction = dict()
ious = []
max_missed = 0
gt_boxes = {}  # Fill this with your ground truth if available

video_path = '/content/drive/MyDrive/VIP CUP/IR_DRONE_139.mp4'
cap = cv2.VideoCapture(video_path)
out = cv2.VideoWriter('output_tracked.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (320, 256))

frame_idx = 0
fps_list = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
    start_time = time.time()

    # Run detection
    results = model(frame)[0]
    detections = []
    for box in results.boxes:
        x1, y1, x2, y2 = [int(coord) for coord in box.xyxy[0]]
        conf = float(box.conf[0])
        cls = int(box.cls[0])
        if cls == 1:  # Only drones (update 0 if your drone class index is different)
            detections.append(([x1, y1, x2 - x1, y2 - y1], conf, cls, None))

    # Run tracking
    tracks = tracker.update_tracks(detections, frame=frame)
    current_ids = set()

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = [int(coord) for coord in ltrb]

        # Direction estimation (based on bbox area change)
        curr_size = (x2 - x1) * (y2 - y1)
        if track_id in track_prev_size:
            prev_size = track_prev_size[track_id]
            if curr_size > prev_size * 1.05:
                direction = "Approaching"
            elif curr_size < prev_size * 0.95:
                direction = "Receding"
            else:
                direction = track_direction.get(track_id, "Stable")
        else:
            direction = "Stable"
        track_prev_size[track_id] = curr_size
        track_direction[track_id] = direction

        # Handle NoneType for conf
        conf = getattr(track, 'det_conf', None)
        conf_str = f'Conf:{conf:.2f}' if conf is not None else 'Conf:-'

        # Draw results
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, f'ID:{track_id}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
        cv2.putText(frame, conf_str, (x1, y2+20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)
        cv2.putText(frame, f'{direction}', (x1, y2+40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

        # IoU calculation if ground truth available
        if frame_idx in gt_boxes:
            iou = compute_iou([x1, y1, x2, y2], gt_boxes[frame_idx])
            ious.append(iou)

        track_last_seen[track_id] = frame_idx
        track_missed.setdefault(track_id, 0)
        current_ids.add(track_id)

    # Update missed frames for tracks not seen in this frame
    for tid in list(track_last_seen.keys()):
        if tid not in current_ids:
            track_missed[tid] += 1
            max_missed = max(max_missed, track_missed[tid])
        else:
            track_missed[tid] = 0

    out.write(frame)
    cv2_imshow(frame)

    elapsed = time.time() - start_time
    fps_list.append(1.0 / elapsed if elapsed > 0 else 0)

    frame_idx += 1
    if frame_idx > 300:  # Limit for Colab demo, remove for full video
        break

cap.release()
out.release()

if ious:
    print(f'Average IoU: {np.mean(ious):.2f}')
else:
    print('Average IoU: N/A')

print(f'Max consecutive missed frames: {max_missed}')

if fps_list:
    print(f'Average FPS: {np.mean(fps_list):.2f}')
else:
    print('Average FPS: N/A')
